In [50]:
#импорт библиотек для работы
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import levene
from scipy.stats import ttest_ind

In [2]:
#чтение и запись данных для работы
orders_df = pd.read_csv('../AB test/ab_orders.csv')
products_df = pd.read_csv('../AB test/ab_products.csv')
users_data_df = pd.read_csv('../AB test/ab_users_data.csv')

#### Предварительный анализ данных по пользователям, заказам и продуктам в заказе

Посмотрим, какие данные находятся в таблице с информацией о пользователях

In [3]:
users_data_df.head()

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37.000000,2022-08-26,1


In [4]:
users_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4337 entries, 0 to 4336
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   4337 non-null   int64 
 1   order_id  4337 non-null   int64 
 2   action    4337 non-null   object
 3   time      4337 non-null   object
 4   date      4337 non-null   object
 5   group     4337 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 203.4+ KB


In [85]:
#изменим формат данных для времени и даты заказа
users_data_df['time'] = pd.to_datetime(users_data_df['time'])
users_data_df['date'] = pd.to_datetime(users_data_df['date'])

In [5]:
#посмотрим количество уникальных пользователей в данных с разбивкой на группы - контрольнаяя (группа 0) и тестовая (1)
users_data_df.groupby('group', as_index=False) \
             .agg({'user_id':'nunique'})

,group,user_id
0,0,515
1,1,502


In [6]:
#посмотрим, какие статусы заказов есть в action
users_data_df.groupby('action', as_index=False) \
             .agg({'user_id':'nunique'})

,action,user_id
0,cancel_order,189
1,create_order,1017


#### Описание данных пользователей

Всего у нас 1 017 уникальных пользователей, которые попали в AB тестирование. В контрольной группе, где пользователи использовали приложение со старой системой рекомендаций, 515 человек. В тестовой группе, где пользователи использовали приложение с новой системой рекомендаций, 502 человека. 

Посмотрим, какие данные находятся в таблице с информацией о заказе

In [7]:
#изучим данные orders_df -  информация о составе заказа
orders_df.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21.000000,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27.000000,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56.000000,"{74, 6}"
4,1259,2022-08-26 00:03:37.000000,"{20, 45, 67, 26}"


In [8]:
#посмотрим на тип данных 
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4123 entries, 0 to 4122
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       4123 non-null   int64 
 1   creation_time  4123 non-null   object
 2   product_ids    4123 non-null   object
dtypes: int64(1), object(2)
memory usage: 96.8+ KB


In [9]:
orders_df['product_ids'] = orders_df.product_ids.apply(lambda x: x.replace('{', '').replace('}', '').split(', '))
orders_df.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"[75, 22, 53, 84]"
1,1256,2022-08-26 00:02:21.000000,"[56, 76, 39]"
2,1257,2022-08-26 00:02:27.000000,"[76, 34, 41, 38]"
3,1258,2022-08-26 00:02:56.000000,"[74, 6]"
4,1259,2022-08-26 00:03:37.000000,"[20, 45, 67, 26]"


In [12]:
#разобьем список продуктов на отдельное значение каждого продукта, чтобы в дальнейшем рассчитать сумму заказа
orders_df = orders_df.explode('product_ids')
orders_df.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,75
0,1255,2022-08-26 00:00:19.000000,22
0,1255,2022-08-26 00:00:19.000000,53
0,1255,2022-08-26 00:00:19.000000,84
1,1256,2022-08-26 00:02:21.000000,56


In [13]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13826 entries, 0 to 4122
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       13826 non-null  int64 
 1   creation_time  13826 non-null  object
 2   product_ids    13826 non-null  object
dtypes: int64(1), object(2)
memory usage: 432.1+ KB


In [14]:
orders_df['product_ids'] = orders_df['product_ids'].astype('int64')

Объединим таблицы с информацией о пользователях и заказах по id заказа для дальнейшей работы

In [15]:
users_data_orders = users_data_df.merge(orders_df, how='inner', on = 'order_id').drop(columns=['creation_time'])
users_data_orders.head()

,user_id,order_id,action,time,date,group,product_ids
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,75
1,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,22
2,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,53
3,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,84
4,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1,56


Посмотрим, какие данные находятся в таблице с информацией о продуктах

In [16]:
products_df.head()

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0


In [17]:
#посмотрим на тип данных 
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  87 non-null     int64  
 1   name        87 non-null     object 
 2   price       87 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ KB


In [19]:
#переименуем колонку product_id для дальнейшего объединения таблиц
products_df = products_df.rename(columns={'product_id':'product_ids'})
#объединим таблицу с информацией о заказе с таблицей с информацией о продукте для рассчета стоимости каждого заказа
sum_product = orders_df.merge(products_df, how='inner', on = 'product_ids')
sum_product = sum_product.groupby('order_id', as_index=False) \
           .agg({'price':'sum'}) \
           .rename(columns={'price':'total_price'})

In [21]:
#объединим таблицу с информацией о пользователях и с суммой заказа
users_orders = users_data_df.merge(sum_product, how='inner', on='order_id')

users_orders.head()

,user_id,order_id,action,time,date,group,total_price
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,408.7
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1,250.5
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0,310.2
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0,85.0
4,966,1258,cancel_order,2022-08-26 00:08:25.486419,2022-08-26,0,85.0


In [22]:
#посмотрим на тип данных 
users_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4337 entries, 0 to 4336
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      4337 non-null   int64  
 1   order_id     4337 non-null   int64  
 2   action       4337 non-null   object 
 3   time         4337 non-null   object 
 4   date         4337 non-null   object 
 5   group        4337 non-null   int64  
 6   total_price  4337 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 271.1+ KB


In [26]:
#в новом датафрейме у нас есть как совершенные заказы, так и отмененные. 
#Для дальнейшего анализа нам нужны только совершенные заказы --> оставим заказы, у которых нет дублей в order_id
users_full = users_orders.drop_duplicates('order_id')
users_full.action.unique()

array(['create_order'], dtype=object)

У нас получилась итоговая таблица users_orders с данными по каждому пользователю, его заказам, дате и времени заказа, итоговая сумма, и к какой группе принадлежит пользователь - контрольной, со старой системой рекомендаций или к тестируемой, с новой системой рекомендаций 

### Метрики и продуктовые гипотезы

Мы будем использовать следующие метрики для оценки качества сервиса
     
1. **Доход на каждого пользователя**
   Мы можем рассчитать данную метрику, так как у нас есть информация о заказах каждого пользователя - заказанные продукты и цена каждой позиции. Это позволяет определить покупательскую способность разных сегментов пользователей приложения и в дальнейшем разрабатывать более индивидуализированные маркетинговые стратегии для каждой группы. Эта метрика вместе с количеством заказов влияет на прибыль, чем выше средний чек, тем скорее выше будет итоговая прибыль. 
   
   **Наша гипотеза 1 - выручка на пользователя при использовании новой рекомендательной системы повысится**
   
   
2. **Количество заказов на 1 пользователя**
   Нам понадобится оценка количества заказов на 1 пользоватея. Это помогает оценить объем использования приложения пользователями, стали ли они заказывать чаще/реже.
   
   **Наша гипотеза 2 - благодаря новой рекомендательной системе количество заказов на 1 пользователя увеличилось** 

#### Рассмотрим статистику по нашим метрикам для пользователей контрольной и тестовых групп 

In [40]:
#разделим данные на две группы - 0 = контрольная, 1 = тестовая группа
test_group = users_full.query('group == 1')
contr_group = users_full.query('group == 0')

#### Средний доход на пользователя

In [54]:
revenue_test = round(test_group.total_price.sum() / test_group.user_id.nunique(), 1)
revenue_contr = round(contr_group.total_price.sum() / contr_group.user_id.nunique(), 1)

diff_revenue = (revenue_test/revenue_contr * 100 - 100).round(1)

print('Выручка на пользователя в контрольной группе',revenue_contr)
print('Выручка на пользователя в тестовой группе',revenue_test)
print('Разница в выручке тестовой и контрольной группы', diff_revenue, 'процентов')

Выручка на пользователя в контрольной группе 1191.2
Выручка на пользователя в тестовой группе 1851.1
Разница в выручке тестовой и контрольной группы 55.4 процентов


Доход на одного пользователя в тестовой группе при использовании нового алгоритма рекомендаций на 55.4% процента выше, чем доход на одного пользователя в контрольной группе при использовании старого алгоритма рекомендаций. При использовании нового алгоритма доход равен 1 851, при использовании старого алгоритма равен 1 191

Необходимо провести статистическую проверку, что полученные данные не являются случайным результатом, а действительно получились после внедрения новой системы рекомендаций. Для сравнения среднего чека по двум группам мы будем использовать t-test, объем наших выборок это позволяет (более 500 наблюдений в каждой группе). Но необходимо проверить на гомогенность наших выборок, чтобы окончательно определить, какой t-тест использовать. 

Уровень значимости будем рассматривать в 5% Нулевая гипотеза при сравнении распределения дисперсий  - дисперсии двух выборок равны, то есть группы имеют гомогенные дисперсии. Альтернативная - обе выборки имеют отличающиеся друг от друга выборки 

In [46]:
statistic, p_value = levene(test_group.total_price, contr_group.total_price)

# Вывод результатов теста
print('Статистика теста Левена:', statistic)
print('p-значение теста Левена:', p_value)

Статистика теста Левена: 0.2630999575712789
p-значение теста Левена: 0.6080258899079423


Полученное значение p-value выше заданного уровня значимости в 5%. Поэтому мы не можем отклонить нулевую гипотезу. В этом случае мы будем использовать t-критерий Стьюдента при проверки гипотезы о разности значения дохода на пользователя

#### Среднее количество заказов на 1 пользователя

In [48]:
order_test = round(test_group.order_id.count() / test_group.user_id.nunique(), 1)
order_contr = round(contr_group.order_id.count() / contr_group.user_id.nunique(), 1)

diff_order = order_test - order_contr

print('Количество заказов на пользователя в контрольной группе',order_contr)
print('Количество заказов в тестовой группе',order_test)
print('Разница в количестве заказов тестовой и контрольной группы', diff_order)

Количество заказов на пользователя в контрольной группе 3.1
Количество заказов в тестовой группе 5.0
Разница в количестве заказов тестовой и контрольной группы 1.9


Количество заказов на 1 пользователя в тестовой группе при использовании новой системы рекомендации увеличилось, чем при использовании старой системы рекомендации - с 3.1 заказа до 5 заказов. 

Предварительно, новая система рекомендаций увеличило метрику количества заказов на пользователя. Но это требует проверки статистическими тестами, можно ли считать данный результат статистически значимым (это случайный результат или новая рекомендательная система действительно помогает увеличить количество заказов на пользователя)

Необходимо провести статистическую проверку, что полученные данные не являются случайным результатом, а действительно получились после внедрения новой системы рекомендаций. Для сравнения среднего количества заказов в день по двум группам мы будем использовать t-test, объем наших выборок это позволяет (более 500 наблюдений в каждой группе). Но необходимо проверить на гомогенность наших выборок, чтобы окончательно определить, какой t-тест использовать.

In [49]:
statistic, p_value = levene(test_group.order_id, contr_group.order_id)

# Вывод результатов теста
print('Статистика теста Левена:', statistic)
print('p-значение теста Левена:', p_value)

Статистика теста Левена: 2.1632113525086822
p-значение теста Левена: 0.14142535735912942


P-значение превышает 5% уровень значимости, мы не можем отвергнуть нашу нулевую гипотезу. В двух выборках наблюдается равенство дисперсий, поэтому мы можем использовать t-критерий Стьюдента для проверки гипотезы

### Статистическая проверка наших продуктовых гипотез

### 1 - новая рекомендательная система предлагает более релевантные товары для клиентов. Благодаря этому выручка на 1 пользователя повысилась

Мы проверим, отличаются ли статистически значимо выручка на 1 пользователя в двух группах. Для этого используем t-критерий Стьюдента. Нулевая гипотеза -  средние значения двух выборок равны. Альтернативная гипотеза - есть статистически значимые различия между группами. Уровень значимости - 5%

In [61]:
revenue_a = test_group.groupby('user_id')['total_price'].sum()
revenue_b = contr_group.groupby('user_id')['total_price'].sum()

In [63]:
statistic, p_value = ttest_ind(revenue_a, revenue_b, equal_var=False)

# Вывод результатов теста
print('Статистика t-критерия Стьюдента:', statistic)
print('p-значение t-критерия Стьюдента:', p_value)

Статистика t-критерия Стьюдента: 11.759608876117142
p-значение t-критерия Стьюдента: 5.6356726300441866e-30


P-значение не превышает 5% уровень значимости, мы можем отвергнуть нашу нулевую гипотезу. В двух выборках средние значения отличаются друг от друга

#### Выводы по 1 гипотезе

1. Наша продуктовая гипотеза о том, что новая система рекомендаций увеличит выручку на 1 пользователя, подтвердилась. Анализ данных показал, что при использовании новой системы выручка на 1 пользователя увеличится на 55.4% - с 1 191 до 1 851. 
2. Статистическая проверка показала, что есть статистически значимая разница между доходом на 1 пользователя при использовании новой системы рекомендаций и доходом на 1 пользователя при использовании старой системы рекомендации. 

**Данный AB тест показал, что использование новой системы рекомендаций увеличит доход на 1 пользователя на 55.4%**

### 2 - Среднее количество заказов на 1 пользователя увеличилось при использовании новой системы рекомендаций

Мы проверим, отличаются ли статистически значимо среднее количество заказов на 1 пользователя в двух группах. Для этого используем t-критерий Стьюдента. Нулевая гипотеза - средние значения двух выборок равны. Альтернативная гипотеза - есть статистически значимые различия между группами. Уровень значимости - 5%

In [65]:
order_a = test_group.groupby('user_id')['order_id'].count()
order_b = contr_group.groupby('user_id')['order_id'].count()

statistic, p_value = ttest_ind(order_a, order_b)

# Вывод результатов теста
print('Статистика t-критерия Стьюдента:', statistic)
print('p-значение t-критерия Стьюдента:', p_value)

Статистика t-критерия Стьюдента: 14.51086812343365
p-значение t-критерия Стьюдента: 1.6974865514796019e-43


P-значение не превышает 5% уровень значимости, мы можем отвергнуть нашу нулевую гипотезу. В двух выборках средние отличаются друг от друга

## Выводы

##### Выводы по 1 гипотезе
1. Наша продуктовая гипотеза о том, что новая система рекомендаций увеличит выручку на 1 пользователя, подтвердилась. Анализ данных показал, что при использовании новой системы выручка на 1 пользователя увеличится на 55.4% - с 1 191 до 1 851. 
2. Статистическая проверка показала, что есть статистически значимая разница между доходом на 1 пользователя при использовании новой системы рекомендаций и доходом на 1 пользователя при использовании старой системы рекомендации. 

Данный AB тест показал, что **использование новой системы рекомендаций увеличит доход на 1 пользователя на 55.4%**

##### Выводы по 2 гипотезе
1. Наша продуктовая гипотеза о том, что новая рекомендательная система увеличила среднее количество заказов на 1 пользователя, подтвердилась на этапе анализа данных. Среднее количество заказов на 1 пользователя увеличилось с 3.1 до 5 заказов.
2. Статистический тест показал, что есть статистически значимая разница между средним количеством заказов на 1 пользователя при использовании новой системы рекомендаций и средним количеством заказов на 1 пользователя при использовании старой системы рекомендации. Мы можем говорить о том, что среднее количество заказов на 1 пользователя увеличивается при использовании новой системы рекомендаций, а не является случайным результатом

Данный AB тест показал, что **использование новой системы рекомендаций увеличит количество заказов на 1 пользователя**